In [ ]:
#Here, load the data

Trying to implement mixup for the preprocessing, as in https://github.com/facebookresearch/mixup-cifar10/blob/main/train.py

In [ ]:
'''def mixup_data(x, y, alpha=1.0, use_cuda=True):
    #Returns mixed inputs, pairs of targets, and lambda
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam'''

In [1]:
#Mixup, as implemented in https://towardsdatascience.com/enhancing-neural-networks-with-mixup-in-pytorch-5129d261bc4a
import numpy as np
import pickle
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
#import torchvision.transforms as transforms Instead, import the audio transforms and use those, so we can use audiomentation and mixup
#at the same time
from torch.utils.data import Dataset, DataLoader

class Mixup(Dataset):
    def __init__(self,data,transform,targets):
        self.transform = transform
        self.data = data
        self.targets = targets
        self.nrow=len(data)

    def __len__(self):
        return np.size(self.data)[0]


    def __getitem__(self, idx):

        # Create a one hot label
        label = torch.zeros(21) #how many do we detect?
        label[self.targets[idx]] = 1.

        # Transform the image by converting to tensor and normalizing it
        if self.transform:
            audio = self.transform(self.data[idx])

        #only perform mixup roughly on 1 for every 5 audios
        if idx > 0 and idx%5 == 0:

            # Choose another audio/label randomly
            mixup_idx = random.randint(0, len(self.data)-1)
            mixup_label = torch.zeros(21)
            label[self.targets[mixup_idx]] = 1.
            if self.transform:
                mixup_audio = self.transform(self.data[mixup_idx])

            # Select a random number from the given beta distribution
            # Mixup the images accordingly
            alpha = 0.2
            lam = np.random.beta(alpha, alpha)
            audio = lam * audio + (1 - lam) * mixup_audio
            label = lam * label + (1 - lam) * mixup_label

        return audio, label
    
#Something like:
'''mixup_data=clone(data)
mixup_labels=clone(labels)
for i in range(len(data)):
    mixup_data[i],mixup_targets[i]=Mixup(data,transform,targets)'''

c:\Users\dalma\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'mixup' is not defined